In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
!pip install evaluate
import evaluate
!pip install rouge_score
import rouge_score
!pip install summa
from summa import keywords



df = pd.read_csv('/content/papers.csv', delimiter=',')
#print(df.head(5))
print(df['pdf_name'][:5])
print(df['paper_text'][:5])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=d20e3630e47d9fffc36b033ee2b1041e6791e32c38926d1a0dec98aca553c892
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54386 sha256=c1eb82062ec647776111a9698a6f02c166f289fe686e1

In [ ]:
print(len(df['paper_text']))
print(len(df['pdf_name']))

7241
7241


In [ ]:

tokenizer = Tokenizer()
train = df['title'].sample(frac=0.8)
test = df['title'].drop(train.index)
#print(train.head())
#print(len(test))
tokenizer.fit_on_texts(train)
total_words = len(tokenizer.word_index) + 1

#print(total_words)

inputs = []

for item in train:
  encoded = tokenizer.texts_to_sequences([item])[0]
  for c in range(1, len(encoded)):
    n_gram_sequence = encoded[:c+1]
    inputs.append(n_gram_sequence)


print(len(inputs))


40550


In [ ]:
max_seq_len = max([len(x) for x in inputs])

input_seq = np.array(pad_sequences(inputs, maxlen = max_seq_len, padding = 'pre'))
#print(input_seq)
#print("test")

predictors = input_seq[:, :-1]

label = input_seq[:, -1]
#print(label)
label = to_categorical(label, num_classes = total_words)
#print(label[0])

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length = max_seq_len - 1))
model.add(LSTM(150))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation = 'softmax'))

model.compile(loss ='categorical_crossentropy', optimizer = 'adam')
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 100)           505000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dropout (Dropout)           (None, 150)               0         
                                                                 
 dense (Dense)               (None, 5050)              762550    
                                                                 
Total params: 1418150 (5.41 MB)
Trainable params: 1418150 (5.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(predictors, label, epochs=25)

Epoch 1/25
1268/1268 [==============================] - 21s 14ms/step - loss: 6.7353
Epoch 2/25
1268/1268 [==============================] - 7s 6ms/step - loss: 6.2293
Epoch 3/25
1268/1268 [==============================] - 7s 5ms/step - loss: 5.8287
Epoch 4/25
1268/1268 [==============================] - 6s 5ms/step - loss: 5.4628
Epoch 5/25
1268/1268 [==============================] - 6s 5ms/step - loss: 5.1312
Epoch 6/25
1268/1268 [==============================] - 6s 5ms/step - loss: 4.8146
Epoch 7/25
1268/1268 [==============================] - 6s 5ms/step - loss: 4.5210
Epoch 8/25
1268/1268 [==============================] - 6s 5ms/step - loss: 4.2417
Epoch 9/25
1268/1268 [==============================] - 6s 4ms/step - loss: 3.9755
Epoch 10/25
1268/1268 [==============================] - 6s 5ms/step - loss: 3.7240
Epoch 11/25
1268/1268 [==============================] - 6s 4ms/step - loss: 3.4949
Epoch 12/25
1268/1268 [==============================] - 6s 5ms/step - loss: 3.2752

In [ ]:
def gen_text(context, next_words, max_seq_len):
  for _ in range(next_words):

    context_list = tokenizer.texts_to_sequences([context])[0]
    context_list = pad_sequences([context_list], maxlen = max_seq_len - 1, padding = 'pre')
    pred_out = model.predict(context_list, verbose = False)[0]
    predicted = np.random.choice(len(pred_out), p=pred_out)

    output = ""

    for word, ind in tokenizer.word_index.items():

      if ind == predicted:
        output += word
        break

    context += " " + output

  return context

In [ ]:
gen_text("Machine Learning", 5, max_seq_len)

'Machine Learning for complex distance selection application'

In [ ]:
!pip install summa
from summa import keywords



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 915.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54386 sha256=1581f0be3cf04230d3d8c05e9610ca87058939304b035ea7b4ab5975e28f770c
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [ ]:
full_text = ' '.join(df['title'])

keys = keywords.keywords(full_text, scores=True)


In [ ]:
print(keys[0:10])

[('learning', 0.3783802898511914), ('learns', 0.3783802898511914), ('learned', 0.3783802898511914), ('learn', 0.3783802898511914), ('models', 0.28699173680580614), ('model', 0.28699173680580614), ('modeling', 0.28699173680580614), ('modelling', 0.28699173680580614), ('networks', 0.22949432721794175), ('network', 0.22949432721794175)]


In [ ]:
import random
context_keys = [x[0] for x in keys[0:30]]
#print(context_keys)

summaries = []
for i in range(0, 200):

  summaries.append(gen_text(context_keys[random.randint(0,29)], 5, max_seq_len))
  #summaries.append(gen_text("", 5, max_seq_len))
  if i % 10 == 0:
    print(i)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190


In [ ]:
print(summaries[0:10])

['models of ocular dominance column formation', 'bases of stochastic gradient methods for', 'bayesian inference for deep neural networks', 'algorithmic stability and generalization performance of', 'neurally inspired plasticity in oculomotor processes', 'algorithm for visual recognition using a', 'algorithm for visual recognition using a', 'optimal learning of the ising model', 'algorithmics and its application to contour', 'modelling seasonality and trends in daily']


In [ ]:
rouge = evaluate.load('rouge')
ref = test.sample(200).tolist()
ref = [x.lower() for x in ref]
#print(ref)
results = rouge.compute(predictions=summaries, references=ref)

print(results)

results_1 = rouge.compute(predictions=summaries, references=train.sample(200).tolist())

print(results_1)

{'rouge1': 0.044579604520007, 'rouge2': 0.000909090909090909, 'rougeL': 0.04358379333514782, 'rougeLsum': 0.04379084739649755}
{'rouge1': 0.04062643902794832, 'rouge2': 0.0, 'rougeL': 0.03719151604329623, 'rougeLsum': 0.03674056451804131}


In [ ]:
summaries_new = []
phrases = ["machine learning", "deep learning", "neural networks", "stochastic gradient", "deep neural networks"]
for phrase in phrases:
  for i in range(0, 20):
    summaries_new.append(gen_text(phrase, 5, max_seq_len))


In [ ]:
ref_1 = test.sample(100).tolist()
for i in range(len(ref_1) - 1):
  ref_1[i] = ref_1[i].lower()
#print(ref_1[0])

results_new = rouge.compute(predictions=summaries_new, references=ref_1)
print(summaries_new[:10])
print(ref_1[:10])
print(results_new)
#print(ref_1[10])

['machine learning rules for cyclic stochastic gradient', 'machine learning for a hybrid nonlinear classification', 'machine learning for feature based reinforcement learning', 'machine learning for graph based semi supervised', 'machine learning applied to perception decision images', "machine learning for 'soft' classification with chemical", 'machine learning for sequences of regularized data', 'machine learning algorithm for estimating object structure', 'machine learning without control uncertainty genetic on', 'machine learning with a three programming constraint']
['exploratory data analysis using radial basis function latent variable models', 'active regression by stratification', 'practical bayesian optimization for model fitting with bayesian adaptive direct search', 'central and pairwise data clustering by competitive neural networks', 'learning from user feedback in image retrieval systems', 'self-organization of hebbian synapses in hippocampal neurons', 'kirchoff law marko

In [ ]:

abstracts = []
title_list = []
for i in range(len(df)):

  if df['abstract'][i] != "Abstract Missing" and len(abstracts) <= 50:
    abstracts.append(df['abstract'][i].lower())
    title_list.append(df['title'][i].lower())

print(len(abstracts))
print(abstracts[0])

51
non-negative matrix factorization (nmf) has previously been shown to 
be a useful decomposition for multivariate data. two different multi- 
plicative algorithms for nmf are analyzed. they differ only slightly in 
the multiplicative factor used in the update rules. one algorithm can be 
shown to minimize the conventional least squares error while the other 
minimizes the generalized kullback-leibler divergence. the monotonic 
convergence of both algorithms can be proven using an auxiliary func- 
tion analogous to that used for proving convergence of the expectation- 
maximization algorithm. the algorithms can also be interpreted as diag- 
onally rescaled gradient descent, where the rescaling factor is optimally 
chosen to ensure convergence. 


In [ ]:
sum_abs = []
for i in range(len(abstracts[0])):
  abs_key = keywords.keywords(abstracts[i], scores=False)
  print(abstracts[i])
  print(abs_key[0])
  print(gen_text(abs_key[0], 5, max_seq_len))
  print()
  #for j in range(0, 5):
  sum_abs.append(gen_text(abs_key[0], 5, max_seq_len))
  #if i % 10 == 0:
  #print(i)
#keys = keywords.keywords(full_text, scores=True)

non-negative matrix factorization (nmf) has previously been shown to 
be a useful decomposition for multivariate data. two different multi- 
plicative algorithms for nmf are analyzed. they differ only slightly in 
the multiplicative factor used in the update rules. one algorithm can be 
shown to minimize the conventional least squares error while the other 
minimizes the generalized kullback-leibler divergence. the monotonic 
convergence of both algorithms can be proven using an auxiliary func- 
tion analogous to that used for proving convergence of the expectation- 
maximization algorithm. the algorithms can also be interpreted as diag- 
onally rescaled gradient descent, where the rescaling factor is optimally 
chosen to ensure convergence. 
f
f estimation in continuous time domains

spike-triggered averaging techniques are effective for linear characterization of neural responses. but neurons exhibit important nonlinear behaviors, such as gain control, that are not captured by such a

KeyboardInterrupt: 

In [ ]:

#print(len(title_list[0].split()))
key_abs = keywords.keywords(abstracts[0], scores=True)
key_abs_list = [x[0] for x in key_abs]
#print(abstracts[0])
print(key_abs_list)
sentence = gen_text(key_abs_list[1], len(title_list[0].split()), max_seq_len)
print(sentence)
print(title_list[0])
#print(rouge.compute(predictions=sentence, references=[title_list[0]]))


['factor', 'matrix factorization', 'rescaling', 'onally rescaled gradient', 'data']
matrix factorization with multiple world graphs a
algorithms for non-negative matrix factorization
